In [69]:
import pandas as pd 
from string import punctuation
import re
from evaluate import load
from IPython import display

In [15]:
df = pd.read_csv('results.csv', index_col=0)
df = df.reset_index(drop=True)

In [16]:
df.shape

(4361, 7)

In [17]:
df.head()

,file,tiny,base,small,medium,large,original_text
0,test_audio/hi/clips/common_voice_hi_26008353.mp3,Hamneya Oscar yiren dheet minaya,حمنے اُس کا یرا ندر من آیا,अमने उसका जरन दिन मनाया,हमने उसका जरन दिन मनाया,हमने उसका जरंदिन मनाया,हमने उसका जन्मदिन मनाया।
1,test_audio/hi/clips/common_voice_hi_26010468.mp3,Sao Delhi never negamsakth shopping mall ke ba...,Sao Delhi Nagarnigam Sakt Shopping Mall ke Bah...,"शाउद दिल्ली नगर निगम सकत, शाबटिग माल के बाहर न...",साउदिल्ली नगर निगम सक्त शॉपिंग मौल के बाहर नही...,साउट दिल्ली नगर निगम सक्त शॉपिंग मौल के बहर नह...,"साउथ दिल्ली नगर निगम सख्त, शॉपिंग मॉल के बाहर ..."
2,test_audio/hi/clips/common_voice_hi_26010469.mp3,Utar koria ne amerika kodi hamle kidam ki,اُدتر کوریع نے امريقا کو دی ہم لیکی دھم کی,उत्तर कोर्या ने आमरेका को दी हमले की दम की,उतर कोर्या ने अमेरिका को दि हम्ले की दम्की,उत्तर कोरिया ने अमेरिका को दी हमले की दम्की,उत्तर कोरिया ने अमेरिका को दी हमले की धमकी
3,test_audio/hi/clips/common_voice_hi_26010470.mp3,"Agle Kamremi, Anek Ruman Murkia hai.","Agle Kamremi, Anik Roman Murthyah hai",अगले कम्रे में अनेक रोमन मुर्तिया हैं,अगले कम्रे में अनेक रोमन मुर्तियां हैं,अगले कम्रे में अनेक रोमन मूर्तियां हैं,अगले कमरे में अनेक रोमन मूर्तियाँ हैं।
4,test_audio/hi/clips/common_voice_hi_26010471.mp3,Thomko ka habizdiya,تم نے Tom ko kaha bhejdia,तुम ने तोम को कहा बेज दिया?,तुमने टोम को कहां भेज दिया?,तुमने टॉम को कहा भेज़ दिया?,तुम ने टॉम को कहाँ भेज दिया?


## Pick up any random sample and see it's results

In [37]:
def pick_any_sample(dataframe):
    random_row = dataframe.sample(n=1)
    print("Original sentence - ", random_row['original_text'].iloc[0])
    print('')
    print("Large Model - ", random_row['large'].iloc[0])
    print("Medium Model - ", random_row['medium'].iloc[0])
    print("Small Model - ", random_row['small'].iloc[0])
    print("Base Model - ", random_row['base'].iloc[0])
    print("Tiny Model - ", random_row['tiny'].iloc[0])
    
    return display.Audio(random_row['file'].iloc[0])

In [77]:
pick_any_sample(df)

Original sentence -  उसने दो किताबें लिखी हैं।

Large Model -  उसने दो कितामें लिखी हैं
Medium Model -  उसने दो कितामें लिखी हैं।
Small Model -  उस्टने दो कितामे लिखी हैं
Base Model -  ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا ھا
Tiny Model -  thus net 2 kTm leki hai


In [78]:
pick_any_sample(df)

Original sentence -  मैं तुम से शादी करूंगी।

Large Model -  मैं तुम से शाथी करूगी
Medium Model -  मैं तुमसे शाथी करूगी।
Small Model -  मैं तुम से शादि करूंगी
Base Model -  मैं तुम से ती करोंटी.
Tiny Model -  I will marry you.


In [81]:
pick_any_sample(df)

Original sentence -  बांग्लादेश में विश्व कप से पहले शुरू हुआ ‘मच्छर मारो अभियान’

Large Model -  बांगलादेश में विश्वकप से पहले शुरू हुआ मच्छर मारो अभियान
Medium Model -  बांगला देश में विश्वकप से पहले शुरू भुआ मच्चर मारो अभियान
Small Model -  बागला देश में विश्वाकप से पहले शुरूवा मच्छर मारू अबियान
Base Model -  بھنگلہ دیش میں ویشوکپ سے پہلے شروع ہوہ مچر مارو ابھیان
Tiny Model -  Bangladesh has been Vishwakup's first show, you are a great maru.


## Calculating WER

In [82]:
wer = load("wer")

In [83]:
100*wer.compute(predictions=df["large"], references=df["original_text"])

64.46587344790937

In [84]:
def remove_punc(text):
    clean_puncts = re.compile(r'[\s{}]+'.format(re.escape(punctuation)))
    cleantext = re.sub(clean_puncts,' ',text)
    cleantext = re.sub('|', '', cleantext)
    return cleantext

In [85]:
df['original_clean'] = df['original_text'].apply(lambda x: remove_punc(x))

In [86]:
100*wer.compute(predictions=df["large"], references=df["original_clean"])

62.61386444341516